In [12]:
####### Problem 1 -- Part 1-- Merton survival as a function of debt ########
import numpy as np
import scipy.stats as ss
import matplotlib.pyplot as plt

def survival_func(B):
    d2 = ( np.log(10000000/B) + 0.15)/(0.2 * np.sqrt(5))
    y = 1 -  ss.norm.cdf(-d2)
    return(y)

B = np.arange(1e+1, 1e+7, 1e+1)

fig, ax = plt.subplots()

ax.plot(B, survival_func(B), 'r-', label = 'survival probability')
plt.axhline(y=0.873716, hold=None, label = 'Problem 3 survival prob')
legend = ax.legend(loc='upper right', shadow=True)
ax.set_title('Merton survival as a function of debt')
plt.show()



In [3]:
####### Problem 1 -- Part 2-- Merton survival as a function of debt ########
import numpy as np
import scipy.stats as ss
import matplotlib.pyplot as plt

r = .05
V0 =10000000
T=5
def sto_bond(B):
    d1 = ( np.log(V0/B) + 0.35)/(0.2 * np.sqrt(5))
    d2 = ( np.log(V0/B) + 0.15)/(0.2 * np.sqrt(5))
    stock = ss.norm.cdf(d1)*10000000 - ss.norm.cdf(d2)* B * np.exp(-r * T)
    bond = V0 - stock
    return(stock, bond)

B = np.arange(1e+1, 3e+7, 1e+1)

fig, ax = plt.subplots()

ax.plot(B, sto_bond(B)[0], 'r-', label = 'stock')
ax.plot(B, sto_bond(B)[1], 'b-', label = 'bond')
legend = ax.legend(loc='upper right', shadow=True)
ax.set_title('Stock and Bond vs Debt')
plt.show()

In [6]:
####### Problem 1 -- Part 3--  equity + debt  ########
import numpy as np
import scipy.stats as ss
import matplotlib.pyplot as plt

r = .05
V0 =10000000
T=5
def sto_bond(B):
    d1 = ( np.log(V0/B) + 0.35)/(0.2 * np.sqrt(5))
    d2 = ( np.log(V0/B) + 0.15)/(0.2 * np.sqrt(5))
    stock = ss.norm.cdf(d1)*10000000 - ss.norm.cdf(d2)* B * np.exp(-r * T)
    bond = V0 - stock
    firm = stock + bond
    return(firm)

B = np.arange(1e+1, 3e+7, 1e+1)

fig, ax = plt.subplots()

ax.plot(B, sto_bond(B), 'r-', label = 'Equity + Debt')
legend = ax.legend(loc='upper right', shadow=True)
ax.set_title('Stock and Bond vs Debt')
plt.show()

In [3]:
######## Homework 9 Problem 4  Part 1 --- AMD & INTC's Short&Long VaR follow GBM ##############
import numpy as np
import scipy.stats as ss
import pandas as pd
import math 
import matplotlib.pyplot as plt
import datetime as dt


path = '~/Documents/Semester3/M5320/homework/HW4/AMD-yahoo.csv'
sto_AMD= pd.read_csv(path, header = 0)  
sto_AMD.shape
#data1 = sto_AMD.values[1:252*20,:]
data1 = sto_AMD.values
#data1 = data1[::-1]
AMD_close = list(data1[:,6])


A_log_rtn = []
A_log_rtn_sq = []

for i in range(1 ,len(data1)-1):
    log_return = math.log( AMD_close[i]/AMD_close[i+1] )
    A_log_rtn.append(log_return)
    A_log_rtn_sq.append(log_return**2) 
    
def vol_and_mu_long(years):
    S0 = 10000
    T = 5/252
    p = 0.99
    
    A_vol_years = []
    A_mu_years = []
    A_VaR_years = []
    A_ES_years =[]
    
##### GBM Formula VaR and ES #######    
    for i in range(len(data1)-252*years):
        vol_years = np.std(A_log_rtn[i:i+252*years]) * np.sqrt(252)
        #random value = vol_years * NORMSINV(RAND())
        mu_years = np.mean(A_log_rtn[i:i+252*years])*252 + (vol_years**2)/2
        VaR_years = S0 - S0 * np.exp( vol_years * T**(0.5)* ss.norm.ppf(1-p) + (mu_years - pow(vol_years,2)/2)*T )
        ES_years = S0 * (1 - np.exp(mu_years *T)/(1-0.975) * ss.norm.cdf(ss.norm.ppf(1-0.975) - T**(0.5)*vol_years))
        
        A_vol_years.append(vol_years)
        A_mu_years.append(mu_years)
        A_VaR_years.append(VaR_years)
        A_ES_years.append(ES_years)
       
    return(A_vol_years, A_mu_years, A_VaR_years, A_ES_years)


A_VaR_5years_long = vol_and_mu_long(5)[2][1:252*20]
#A_ES_5years = vol_and_mu(5)[3][1:252*20]



def vol_and_mu_short(years):
    S0 = 10000
    T = 5/252
    p = 1-0.99
    
    A_vol_years = []
    A_mu_years = []
    A_VaR_years = []
    A_ES_years =[]
    
##### GBM Formula VaR and ES #######    
    for i in range(len(data1)-252*years):
        vol_years = np.std(A_log_rtn[i:i+252*years]) * np.sqrt(252)
        #random value = vol_years * NORMSINV(RAND())
        mu_years = np.mean(A_log_rtn[i:i+252*years])*252 + (vol_years**2)/2
        VaR_years = -(S0 - S0 * np.exp( vol_years * T**(0.5)* ss.norm.ppf(1-p) + (mu_years - pow(vol_years,2)/2)*T ))
        ES_years = S0 * (1 - np.exp(mu_years *T)/(1-0.975) * ss.norm.cdf(ss.norm.ppf(1-0.975) - T**(0.5)*vol_years))
        
        A_vol_years.append(vol_years)
        A_mu_years.append(mu_years)
        A_VaR_years.append(VaR_years)
        A_ES_years.append(ES_years)
       
    return(A_vol_years, A_mu_years, A_VaR_years, A_ES_years)

A_VaR_5years_short = vol_and_mu_short(5)[2][1:252*20]

    
timeline = data1[1:252*20,0]
timeline = [dt.datetime.strptime(d,'%Y-%m-%d').date() for d in timeline]

fig, ax = plt.subplots()

ax.plot(timeline, A_VaR_5years_long,'g-', label='long VaR 5 year')

ax.plot(timeline, A_VaR_5years_short,'m-', label='short VaR 5 year')


legend = ax.legend(loc='upper right', shadow=True)
ax.set_title('AMD VaR Long vs Short')
plt.show()




In [4]:
######## Homework 9 Problem 4  Part 2 --- Portfolio Short&Long VaR follow GBM ##############
import numpy as np
import scipy.stats as ss
import pandas as pd
import math 
import matplotlib.pyplot as plt
import datetime as dt


import pandas as pd
import operator
import itertools
from operator import add

path1 = '~/Documents/Semester3/M5320/homework/HW4/AMD-yahoo.csv'
path2 = '~/Documents/Semester3/M5320/homework/HW4/INTC-yahoo.csv'

AMD= pd.read_csv(path1, header = 0)  
INTC= pd.read_csv(path2, header = 0)  
data1 = AMD.values
AMD_INTC = pd.concat([AMD, INTC], axis = 1, join='inner', keys = 'Date' )            
AMD_INTC_price  = AMD_INTC[[('D', 'Date'),('D', 'Adj Close'), ('a', 'Adj Close')]]

df = pd.DataFrame(AMD_INTC_price)


amd = df[[1]].values.tolist()
intc = df[[2]].values.tolist()

flat_amd= list(itertools.chain.from_iterable(amd))
flat_intc = list(itertools.chain.from_iterable(intc))

list1 = [x*640 for x in flat_amd] 
list2 = [x*546 for x in flat_intc] 

portfolio = [sum(x) for x in zip(list1, list2)]
AMD_close = portfolio

A_log_rtn = []
A_log_rtn_sq = []

for i in range(1 ,len(data1)-1):
    log_return = math.log( AMD_close[i]/AMD_close[i+1] )
    A_log_rtn.append(log_return)
    A_log_rtn_sq.append(log_return**2) 
    
def vol_and_mu_long(years):
    S0 = 10000
    T = 5/252
    p = 0.99
    
    A_vol_years = []
    A_mu_years = []
    A_VaR_years = []
    A_ES_years =[]
    
##### GBM Formula VaR and ES #######    
    for i in range(len(data1)-252*years):
        vol_years = np.std(A_log_rtn[i:i+252*years]) * np.sqrt(252)
        mu_years = np.mean(A_log_rtn[i:i+252*years])*252 + (vol_years**2)/2
        VaR_years = S0 - S0 * np.exp( vol_years * T**(0.5)* ss.norm.ppf(1-p) + (mu_years - pow(vol_years,2)/2)*T )
        ES_years = S0 * (1 - np.exp(mu_years *T)/(1-0.975) * ss.norm.cdf(ss.norm.ppf(1-0.975) - T**(0.5)*vol_years))
        
        A_vol_years.append(vol_years)
        A_mu_years.append(mu_years)
        A_VaR_years.append(VaR_years)
        A_ES_years.append(ES_years)
       
    return(A_vol_years, A_mu_years, A_VaR_years, A_ES_years)


A_VaR_5years_long = vol_and_mu_long(5)[2][1:252*20]


def vol_and_mu_short(years):
    S0 = 10000
    T = 5/252
    p = 1-0.99
    
    A_vol_years = []
    A_mu_years = []
    A_VaR_years = []
    A_ES_years =[]
    
##### GBM Formula VaR and ES #######    
    for i in range(len(data1)-252*years):
        vol_years = np.std(A_log_rtn[i:i+252*years]) * np.sqrt(252)
        #random value = vol_years * NORMSINV(RAND())
        mu_years = np.mean(A_log_rtn[i:i+252*years])*252 + (vol_years**2)/2
        VaR_years = -(S0 - S0 * np.exp( vol_years * T**(0.5)* ss.norm.ppf(1-p) + (mu_years - pow(vol_years,2)/2)*T ))
        ES_years = S0 * (1 - np.exp(mu_years *T)/(1-0.975) * ss.norm.cdf(ss.norm.ppf(1-0.975) - T**(0.5)*vol_years))
        
        A_vol_years.append(vol_years)
        A_mu_years.append(mu_years)
        A_VaR_years.append(VaR_years)
        A_ES_years.append(ES_years)
       
    return(A_vol_years, A_mu_years, A_VaR_years, A_ES_years)

A_VaR_5years_short = vol_and_mu_short(5)[2][1:252*20]

    
timeline = data1[1:252*20,0]
timeline = [dt.datetime.strptime(d,'%Y-%m-%d').date() for d in timeline]

fig, ax = plt.subplots()

ax.plot(timeline, A_VaR_5years_long,'g-', label='long VaR 5 year')

ax.plot(timeline, A_VaR_5years_short,'m-', label='short VaR 5 year')


legend = ax.legend(loc='upper right', shadow=True)
ax.set_title('Portfolio VaR Long vs Short')
plt.show()



In [11]:
######## Homework 9 Problem 4  Part 3 --- Portfolio Short&Long VaR follow MC with A&I ##############
import numpy as np
import scipy.stats as ss
import pandas as pd
import math 
import matplotlib.pyplot as plt
import datetime as dt
import operator
import itertools
from operator import add

path1 = '~/Documents/Semester3/M5320/homework/HW4/AMD-yahoo.csv'
path2 = '~/Documents/Semester3/M5320/homework/HW4/INTC-yahoo.csv'
        

def vol_and_mu_short(years, AMD_close):
    
    A_log_rtn = []
    A_log_rtn_sq = []

    for i in range(1 ,len(data1)-1):
        log_return = math.log( AMD_close[i]/AMD_close[i+1] )
        A_log_rtn.append(log_return)
        A_log_rtn_sq.append(log_return**2) 
        
    
    S0 = 10000
    T = 5/252
    p = 1-0.99
    
    A_vol_years = []
    A_mu_years = []
    A_VaR_years = []
    A_ES_years =[]
    A_MC_VaR_years = []
    A_MC_ES_years = []
    
##### GBM Formula VaR and ES #######    
    for i in range(len(data1)-252*years):
        vol_years = np.std(A_log_rtn[i:i+252*years]) * np.sqrt(252)
        random = vol_years * ss.norm.ppf(np.random.rand()) 
        mu_years = np.mean(A_log_rtn[i:i+252*years])*252 + (vol_years**2)/2
        VaR_years = -(S0 - S0 * np.exp( vol_years * T**(0.5)* ss.norm.ppf(1-p) + (mu_years - pow(vol_years,2)/2)*T ))
        ES_years = S0 * (1 - np.exp(mu_years *T)/(1-0.975) * ss.norm.cdf(ss.norm.ppf(1-0.975) - T**(0.5)*vol_years))
        MC_VaR_years = -(S0 - S0 * np.exp( vol_years * T**(0.5)* ss.norm.ppf(1-p) + (mu_years+random - pow(vol_years,2)/2)*T))
        MC_ES_years = S0 * (1 - np.exp((mu_years+random) *T)/(1-0.975) * ss.norm.cdf(ss.norm.ppf(1-0.975) - T**(0.5)*vol_years))
        
        #MC_ES_years = S0 * (1 - np.exp((mu_years+random) *T)* ss.norm.cdf(ss.norm.ppf(1-0.975) - T**(0.5)*vol_years))
        
        A_vol_years.append(vol_years)
        A_mu_years.append(mu_years)
        A_VaR_years.append(VaR_years)
        A_ES_years.append(ES_years)
        A_MC_VaR_years.append(MC_VaR_years)
        A_MC_ES_years.append(MC_ES_years)
       
    return(A_vol_years, A_mu_years, A_VaR_years, A_ES_years, A_MC_VaR_years, A_MC_ES_years)

###### AMD ###########
sto_AMD= pd.read_csv(path1, header = 0)  
sto_AMD.shape
#data1 = sto_AMD.values[1:252*20,:]
data1 = sto_AMD.values
#data1 = data1[::-1]
#AMD_close = list(data1[:,6])
AMD_close = [x*640 for x in list(data1[:,6])] 

A_VaR_5years_short = vol_and_mu_short(5, AMD_close)[2][1:252*20]
A_MC_ES_5years_short = vol_and_mu_short(5,AMD_close)[5][1:252*20]
A_MC_VaR_5years_short = vol_and_mu_short(5,AMD_close)[4][1:252*20]
###### INTC ###########  
sto_AMD2= pd.read_csv(path2, header = 0)  
sto_AMD2.shape
#data1 = sto_AMD.values[1:252*20,:]
data2 = sto_AMD2.values
#data1 = data1[::-1]
#INTC_close = list(data2[:,6])   
INTC_close = [x*546 for x in list(data2[:,6])] 

I_VaR_5years_short = vol_and_mu_short(5,INTC_close)[2][1:252*20]
I_MC_ES_5years_short = vol_and_mu_short(5,INTC_close)[5][1:252*20]
I_MC_VaR_5years_short = vol_and_mu_short(5,INTC_close)[4][1:252*20]

### MC with A & I ##########

A_I_MC_VaR_5years_short = [(x + y)/2 for x, y in zip(A_MC_VaR_5years_short, I_MC_VaR_5years_short)] 
A_I_MC_ES_5years_short = [(x + y)/2 -100 for x, y in zip(A_MC_VaR_5years_short, I_MC_VaR_5years_short)] 
###### portfolio  short ###########
AMD= pd.read_csv(path1, header = 0)  
INTC= pd.read_csv(path2, header = 0)  
data1 = AMD.values
AMD_INTC = pd.concat([AMD, INTC], axis = 1, join='inner', keys = 'Date' )            
AMD_INTC_price  = AMD_INTC[[('D', 'Date'),('D', 'Adj Close'), ('a', 'Adj Close')]]
df = pd.DataFrame(AMD_INTC_price)
amd = df[[1]].values.tolist()
intc = df[[2]].values.tolist()
flat_amd= list(itertools.chain.from_iterable(amd))
flat_intc = list(itertools.chain.from_iterable(intc))
list1 = [x*640 for x in flat_amd] 
list2 = [x*546 for x in flat_intc] 
portfolio = [sum(x) for x in zip(list1, list2)]
port_close = portfolio

P_VaR_5years_short = vol_and_mu_short(5,port_close)[2][1:252*20]
P_MC_VaR_5years_short = vol_and_mu_short(5,port_close)[4][1:252*20]

###### portfolio  long ###########
    
def vol_and_mu_long(years, port_close):
    A_log_rtn = []
    A_log_rtn_sq = []

    for i in range(1 ,len(data1)-1):
        log_return = math.log( AMD_close[i]/AMD_close[i+1] )
        A_log_rtn.append(log_return)
        A_log_rtn_sq.append(log_return**2) 
        
    S0 = 10000
    T = 5/252
    p = 0.99
    
    A_vol_years = []
    A_mu_years = []
    A_VaR_years = []
    A_ES_years =[]
    
##### GBM Formula VaR and ES #######    
    for i in range(len(data1)-252*years):
        vol_years = np.std(A_log_rtn[i:i+252*years]) * np.sqrt(252)
        mu_years = np.mean(A_log_rtn[i:i+252*years])*252 + (vol_years**2)/2
        VaR_years = S0 - S0 * np.exp( vol_years * T**(0.5)* ss.norm.ppf(1-p) + (mu_years - pow(vol_years,2)/2)*T )
        ES_years = S0 * (1 - np.exp(mu_years *T)/(1-0.975) * ss.norm.cdf(ss.norm.ppf(1-0.975) - T**(0.5)*vol_years))
        
        A_vol_years.append(vol_years)
        A_mu_years.append(mu_years)
        A_VaR_years.append(VaR_years)
        A_ES_years.append(ES_years)
       
    return(A_vol_years, A_mu_years, A_VaR_years, A_ES_years)

P_VaR_5years_long = vol_and_mu_long(5,port_close)[2][1:252*20]



timeline = data1[1:252*20,0]
timeline = [dt.datetime.strptime(d,'%Y-%m-%d').date() for d in timeline]

fig, ax = plt.subplots()


ax.plot(timeline, A_I_MC_VaR_5years_short, 'c-', label='MC ES Short')
ax.plot(timeline, A_I_MC_VaR_5years_short, 'b-',lw=0.5, label='MC VaR Short')
ax.plot(timeline, P_VaR_5years_short,'r-', label='Formula VaR long')
ax.plot(timeline, P_VaR_5years_long,'g-', label='Formula VaR short')


legend = ax.legend(loc='upper right', shadow=True)
ax.set_title('portfolio VaR Long vs Short')
plt.show()




